In [4]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer #Для преобразования входных данных
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer #Для изменения данных в столбце или в строке
from sklearn.preprocessing import OneHotEncoder
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error

X_full = pd.read_csv('D:/Python_Projects/Pipeline_Datasets/Train.csv', index_col='ID')
X_test_full = pd.read_csv('D:/Python_Projects/Pipeline_Datasets/Test.csv', index_col='ID')

print(X_full.columns) #выведет все столбцы в таблице
print(X_full.sample(5)) #выведет несколько случайных строк из таблицы
X_full.describe() #описание dataset

Index(['winery', 'wine', 'year', 'rating', 'num_reviews', 'country', 'region',
       'price', 'type', 'body', 'acidity'],
      dtype='object')
                  winery                   wine  year  rating  num_reviews  \
ID                                                                           
5028           Clos Pons                 Roc Nu  2011     4.2          420   
2034    Bodegas La Horra              Corimbo I  2015     4.2          412   
4923         Matarromera           Gran Reserva  2011     4.2          389   
2564  Portal del Priorat  Clos del Portal Somni  2014     4.2          404   
1259        Bodegas Roda     Roda Reserva Rioja  2004     4.3          452   

     country             region  price                  type  body  acidity  
ID                                                                           
5028  Espana  Costers del Segre  23.95                   Red   NaN      NaN  
2034  Espana   Ribera del Duero  37.90  Ribera Del Duero Red   5.0      3.

,rating,num_reviews,price,body,acidity
count,5998.000000,5998.000000,5993.000000,5066.000000,5066.000000
mean,4.268690,462.406469,66.930932,4.175484,2.944137
std,0.128355,808.076299,165.434281,0.582908,0.258017
min,4.200000,25.000000,4.990000,2.000000,1.000000
25%,4.200000,384.000000,19.900000,4.000000,3.000000
50%,4.200000,402.000000,29.900000,4.000000,3.000000
75%,4.300000,415.000000,61.940000,5.000000,3.000000
max,4.900000,32624.000000,3119.080000,5.000000,3.000000


In [6]:
import pandas as pd
X_full = pd.read_csv('D:/Python_Projects/Pipeline_Datasets/Train.csv', index_col='ID')
X_test_full = pd.read_csv('D:/Python_Projects/Pipeline_Datasets/Test.csv', index_col='ID')

# Удаляем из датасета price, т.к. мы не можем использовать price для обучения, т.к. у нас цель научиться предсказывать price

X_full.dropna(axis=0, subset=['price'], inplace=True) #Сначала удаляем все значения, где не указана price,
#inplace=True - меняем значения в этой же переменной, axis=0 - удаляем строки где price nan

# Разбиваем датасет на цель (y) и входные данные (X)
y = X_full.price #удаляем price из X_full чтобы price сохранилась в переменной цели

X_full.drop(['price'], axis=1, inplace=True) #удаляем из X_full колонку с price, и изменения применяем в эту же переменную

# Разбиваем датасет на train/val
X_train_full, X_valid_full, y_train, y_valid = train_test_split(X_full, y, 
                                                                train_size=0.8, test_size=0.2,
                                                                random_state=0) 
#random_state=0 параметр для перемешивания данных, чтобы в следующиц раз этот датасет также перемешался

In [7]:
# В наших данных есть нечисловые значения, которыe затрудняют обработку, поэтому оставляем только те данные, где нечисловые 
#значения имеют меньше 10 разных значений (для удобства дальнейшего кодирования)
categorical_cols = [cname for cname in X_train_full.columns if
                    X_train_full[cname].nunique() < 10 and 
                    X_train_full[cname].dtype == "object"]

# ...и оставляем все числовые данные
numerical_cols = [cname for cname in X_train_full.columns if 
                X_train_full[cname].dtype in ['int64', 'float64']]

# Оставляем в dataset только те колонки, с которыми хотим работать дальше
my_cols = categorical_cols + numerical_cols
X_train = X_train_full[my_cols].copy()
X_valid = X_valid_full[my_cols].copy()
X_test = X_test_full[my_cols].copy()

print(len(X_train.columns), len(X_full.columns))
X_train.head()

5 10


,country,rating,num_reviews,body,acidity
ID,,,,,
4282,Espana,4.2,414,3.0,3.0
5024,Espana,4.2,388,5.0,3.0
2454,Espana,4.2,420,4.0,3.0
3767,Espana,4.2,400,5.0,3.0
3557,Espana,4.2,414,3.0,3.0


In [8]:
# Pipeline - спасает от необходимости не забыть применить теже самые преобразования к train и val
# Подготовка данных

# Инициализируем препроцессор для заполнения недостающих числовых значений

numerical_transformer = SimpleImputer(strategy='constant') #constant заполнит все недостающие данные одним числом,по умолчанию 
#это 0

#Инициализируем препроцессор для заполнения недостающих нечисловых значений,добавляем кодирование нечисловых значений в числовые

categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))]) 
#pipneline берет два разных действия и выполняет их по проядку

#handle_unknown='ignore', если в данных появится новое строковое значение, то onehotencoder выбросит ощибку обработки, т.к.
#не будет знать что с этим значением делать, handle_unknown ='ignore' позволит при появление нового строкового значения его 
#обрабатывать, т.е. значение будет закодировано в что-то похожее на '000' 

# Создаем общий препроцессор данных

preprocessor = ColumnTransformer(
    transformers=[
        ('numerical', numerical_transformer, numerical_cols),
        ('categorical', categorical_transformer, categorical_cols)
    ])

In [11]:
# Model
# Инициализируем модель
model = RandomForestRegressor(n_estimators=200, random_state=0)

# Создаем пайплайн из модели и препроцессора
classifier = Pipeline(steps=[('preprocessor', preprocessor), ('model', model)], verbose=True)
#данные будут уходить на preprocessor а потом передаваться на model

# Обучение
# Передаем обучающие данные в пайплайн, и обучаем
classifier.fit(X_train, y_train)

[Pipeline] ...... (step 1 of 2) Processing preprocessor, total=   0.0s
[Pipeline] ............. (step 2 of 2) Processing model, total=   0.8s


Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('numerical',
                                                  SimpleImputer(strategy='constant'),
                                                  ['rating', 'num_reviews',
                                                   'body', 'acidity']),
                                                 ('categorical',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('onehot',
                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                  ['country'])])),
                ('model',
                 RandomForestRegressor(n_estimators=200, random_state=0))],
         verbose=True)

In [12]:
# Оценка модели
# Передаем валидационные данные в пайплайн, получаем предсказания
preds = classifier.predict(X_valid)

# Оцениваем точность модели
print('MAE:', mean_absolute_error(y_valid, preds)) 
#MAE - средняя ошибка при предсказании цены

MAE: 35.7237335626876


In [24]:
# Смотрим цену вина в 50 строке
wine_to_check = 50
print(X_valid.iloc[wine_to_check].to_string())
print()
print(y_valid.iloc[wine_to_check])

country        Espana
rating            4.2
num_reviews       420
body              4.0
acidity           3.0

28.3


In [26]:
classifier.predict(X_valid.iloc[wine_to_check:wine_to_check+1]) # Стоимость вина по результатм предсказания

array([22.10124597])